# Imports


In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Model Building
import ultralytics
from ultralytics import YOLO
import labelme2yolo

# Others
import os
import shutil
import zipfile

%matplotlib inline

In [2]:
print(os.getcwd())
print(os.listdir('.'))

t:\EY_Challenge\EY_Challenge\Rough Work
['Footprint_Data.ipynb', 'Yolo_WIth_PrePost.ipynb']


# Download the Datasets

In [5]:
# Uncomment what you need

# !wget https://challenge.ey.com/api/v1/storage/admin-files/Post_Event_San_Juan.tif -O ../Datasets/Post_Event_San_Juan.tif
# !wget https://challenge.ey.com/api/v1/storage/admin-files/Pre_Event_San_Juan.tif -O ../Datasets/Pre_Event_San_Juan.tif
# !wget https://challenge.ey.com/api/v1/storage/admin-files/027282564879278492-65c1f5242482662856c6d3ee-Buildins%20Footprint%20ROI.zip -O ../Datasets/Building_Footprint_ROI.zip

--2024-02-17 17:05:12--  https://challenge.ey.com/api/v1/storage/admin-files/027282564879278492-65c1f5242482662856c6d3ee-Buildins%20Footprint%20ROI.zip
Resolving challenge.ey.com (challenge.ey.com)... 52.236.158.32
Connecting to challenge.ey.com (challenge.ey.com)|52.236.158.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28036831 (27M) [application/octet-stream]
Saving to: '../Datasets/Building_Footprint_ROI.zip'

     0K .......... .......... .......... .......... ..........  0% 2.28M 12s
    50K .......... .......... .......... .......... ..........  0% 4.56M 9s
   100K .......... .......... .......... .......... ..........  0%  212M 6s
   150K .......... .......... .......... .......... ..........  0% 54.9M 5s
   200K .......... .......... .......... .......... ..........  0% 4.04M 5s
   250K .......... .......... .......... .......... ..........  1% 4.60M 5s
   300K .......... .......... .......... .......... ..........  1% 67.4M 4s
   350K .......... 

In [6]:
# Use this to extract the building footprint dataset

zip_path = '../Datasets/Building_Footprint_ROI.zip'
extract_path = '../Datasets/Building_Footprint_ROI'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [3]:
pre_event_image = '../Datasets/Pre_Event_San_Juan.tif'
post_event_image = '../Datasets/Post_Event_San_Juan.tif'
footprint_data = '../'


## Load the Datasets & Tile

In [7]:
def generate_tiles(input_file, output_dir,grid_x,grid_y):
    ds = gdal.Open(input_file)

    # Get image size and number of bands
    width = ds.RasterXSize
    height = ds.RasterYSize
    num_bands = ds.RasterCount

    # Calculate number of tiles in each dimension
    num_tiles_x = (width // grid_x)
    num_tiles_y = (height // grid_y)

    print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over each tile and save as a separate TIFF image
    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            x_offset = i *  grid_x
            y_offset = j *  grid_y

            tile_width = min(grid_x, width - x_offset)
            tile_height = min(grid_y, height - y_offset)

            tile = []
            for band in range(1, num_bands + 1):
                tile_data = ds.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
                tile.append(tile_data)

            # Create output filename
            output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif")
                
            # Create an output TIFF file with same CRS and band values range
            driver = gdal.GetDriverByName("GTiff")
            options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']
            out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, 
                       ds.GetRasterBand(1).DataType, options=options)
            # out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, ds.GetRasterBand(1).DataType)

            # Set the geotransform
            geotransform = list(ds.GetGeoTransform())
            geotransform[0] = geotransform[0] + x_offset * geotransform[1]
            geotransform[3] = geotransform[3] + y_offset * geotransform[5]
            out_ds.SetGeoTransform(tuple(geotransform))

            # Set the projection
            out_ds.SetProjection(ds.GetProjection())

            # Write each band to the output file
            for band in range(1, num_bands + 1):
                out_band = out_ds.GetRasterBand(band)
                out_band.WriteArray(tile[band - 1])

            # Close the output file
            out_ds = None

    print("Tiles generation completed.")

In [ ]:
input_file = '../Datasets/Post_Event_San_Juan.tif'
output_dir = "../Post_Event_Grids_In_TIFF"
grid_x = 512
grid_y = 512
generate_tiles(input_file, output_dir,grid_x,grid_y)